In [1]:
!pip install feature-engine

# Importing all required libraries
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.offline as pyo
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import scipy.stats as stats
import statistics
import re

from feature_engine.transformation import PowerTransformer
from feature_engine.imputation import RandomSampleImputer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek

from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
import xgboost
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import plot_tree, export_text

from sklearn.model_selection import GridSearchCV


from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [2]:
dataset = pd.read_csv('../input/titanic/train.csv')
print(dataset.shape)

dataset.head()

> Feature Engineering: Feature engineering is one of the important techniques to improve our model accuracy. This technique uses the data to it's maximum to create new features from the existing features in our training dataset. 
> 
> There is a scope for feature engineering in our dataset.
> > For example: We can extract data and create new features from : 'Name' and 'Ticket' columns. (Go to the Feature Engineering section for practical view)

# The Target Variable

> I used both plotly and matplot in this notebook.

In [3]:
data = [go.Bar(x = ['Not Survived', 'Survived'], y = dataset['Survived'].value_counts(), marker=dict(color='#CD7F32'))]
layout = go.Layout(title = 'Titanic Dataset: Target Feature')

fig = go.Figure(data, layout)
fig.update_layout(autosize = False, width = 500, height = 500, yaxis_title = 'Count')

pyo.iplot(fig)


> Hover over the plot to see the exact number of people for each category.

In [4]:
(dataset['Survived'].value_counts() / len(dataset['Survived'])).apply(lambda x: round(x*100, 2))

> This is an imbalanced dataset. We can see that 61.62% of values tell us about who didn't survive the tragic titanic incident.
> > Inorder to make this a balanced dataset, we have to make synthetic samples using methods such as oversampling the minority class (i.e., class-1), undersampling the majority class (i.e., class-0) or performing both over and under sampling methods.


# The Variable Types

> Checking the data type of all features is an important step before proceeding further. It is possible for pandas to make incorrect assumptions about the dataset while loading.
> > In this dataset, we can see two features dtype is incorrect. 

In [5]:
dataset.info()

> 'Survived' is our target feature. Classification algorithms work only when the target feauture is a cateogry. Hence, we must convert Survived dtype.

> As per the given data_description.txt file, Pclass is a category, pandas dataframe read figured it as an int64 type. We must convert it.

## Datatype Conversion

> Converting datatypes of 'Survived' and 'Pclass' features to object. Since they are not integers.

In [6]:
dataset['Survived'] = dataset['Survived'].astype('object')
dataset['Pclass'] = dataset['Pclass'].astype('object')

In [7]:
categorical_features = [feature for feature in dataset.columns if dataset[feature].dtype == 'O' and feature != 'Survived']
print("Total number of Categorical Features: ", len(categorical_features))

print(categorical_features)

In [8]:
numerical_features = [feature for feature in dataset.columns if feature not in categorical_features and feature != 'Survived']
print("Total number of Numerical Features: ", len(numerical_features))

print(numerical_features)

# Missing Values

> Missing values creates a bias in our dataset that effects the performance of our model. It reduces the statistical power of accepting and rejecting the null hypothesis and also representativeness of the samples.

> We can drop the entire feature if we have high percentage of missing values in it to avoid biased results by imputation methods.

In [9]:
features_with_missing_values = [feature for feature in dataset.columns if dataset[feature].isnull().sum() > 0]
print('Total number of Numerical features which have null values: ', len(features_with_missing_values))

print(features_with_missing_values)

##  Visualizing the percentage of missing values

In [10]:
feature_names = list((dataset[features_with_missing_values].isnull().mean().sort_values(ascending=False)).apply(lambda x: x*100).index)
percentage_of_na = (dataset[features_with_missing_values].isnull().mean().sort_values(ascending=False)).apply(lambda x: x*100)

data = [go.Bar(x = feature_names, y = percentage_of_na, marker=dict(color='#CD7F32'))]
layout = go.Layout(title = 'Percentage of missing values in each feature')

fig = go.Figure(data, layout)
fig.update_layout(autosize = False, width = 600, height = 550, yaxis_title = 'Percentage')

pyo.iplot(fig)

> About 77.1% of Cabin values are missing. We can drop this feature.

> About 19.86% of Age values are missing, We will check the distribution and decide what to do with it.

> Since there are only two rows(0.22%) of missing data for Embarked column, we can drop those rows.

## Relationship between the missing values and target variable

In [11]:
def analyse_missing_data(df, feature):
    data = df.copy()
    
    # 1 -> If value is missing and 0 -> If value is not missing
    data[feature] = np.where(data[feature].isnull(), 1, 0)
    temp_data = data.groupby(feature)['Survived'].agg('count')
    
    data = [go.Bar(y = temp_data, x = ['Not Missing', 'Missing'])]
    layout = go.Layout(title = 'Relationship between {} and Survived'.format(feature))
    
    fig = go.Figure(data, layout)
    fig.update_layout(autosize = False, width = 600, height = 400, yaxis_title = feature)
    
    pyo.iplot(fig)
    

In [12]:
for feature in features_with_missing_values:
    analyse_missing_data(dataset, feature)

## Numerical Features

### Age

In [13]:
def plot_hist(feature):
    fig, ax = plt.subplots(2, 1, figsize=(10, 8))
    
    sns.histplot(data = dataset[feature], kde = True, ax = ax[0])

    ax[0].axvline(x = dataset[feature].mean(), color = 'r', linestyle = '--', linewidth = 2, label = 'Mean: {}'.format(round(dataset[feature].mean(), 3)))
    ax[0].axvline(x = dataset[feature].median(), color = 'orange', linewidth = 2, label = 'Median: {}'.format(round(dataset[feature].median(), 3)))
    ax[0].axvline(x = statistics.mode(dataset[feature]), color = 'yellow', linewidth = 2, label = 'Mode: {}'.format(statistics.mode(dataset[feature])))
    ax[0].legend()
    
    sns.boxplot(x = dataset[feature], ax = ax[1])
    
    plt.show()

plot_hist('Age')

> Age distribution is right skewed, we could apply few transformations (after imputing missing values) and try to make it closer to normal distribution.

#### End-of-tail Imputation

> End of tail Imputation technique is generally used when more data points are located at the end of distribution. 

> > For **normally distributed** feature, we can simply use the below formula:
> > >  ##### mean $\pm$ 3 * std 

> > For **skewed distribution**, we can use IQR proximity rule:
> > >  ##### IQR = 75$^{th}$ Quantile - 25$^{th}$ Quantile
> > >  ##### Upper Limit = 75$^{th}$ Quantile + IQR x 1.5
> > >  ##### Lower Limit = 25$^{th}$ Quantile - IQR x 1.5
> > > > **Note**: For extreme outliers, multiply IQR with 3 instead of 1.5




In [14]:
quantile_0_75 = dataset['Age'].quantile(0.75)
quantile_0_25 = dataset['Age'].quantile(0.25)

IQR = quantile_0_75 - quantile_0_25
upper_limit = quantile_0_75 + (IQR * 1.5)
lower_limit = quantile_0_25 - (IQR * 1.5)

dataset['Age_IQR'] = dataset['Age'].fillna(upper_limit)
plot_hist('Age_IQR')

> We are creating two modes by using this technique.

#### Random Sample Imputation

> Random sample imputation draws the observations from the original distribution of the feature and fills it in nan values. This technique is suitable for both numerical and categorical features.

> It is easy to implement using **.sample()** method on pandas dataframe. More importantly, it preserves the variance of the feature. To avoid randomness we must set a seed.

In [15]:
def random_sample_imputer(feature, random_state):

    dataset[feature + '_random'] = dataset[feature]

    random_sample_age = dataset[feature].dropna().sample(dataset[feature].isnull().sum(), random_state = random_state)
    random_sample_age.index = dataset[dataset[feature].isnull()].index

    dataset.loc[dataset[feature].isnull(), feature + '_random'] = random_sample_age
    
    return dataset

dataset = random_sample_imputer('Age', 1111)
plot_hist('Age_random')

#### Applying Transformations

> To avoid code repetition, I have created the below methods to get the **estimates of location** (Mean, Median and Mode) and apply **five different transformations** (Yeojohnson, Boxcox, Log, Power and Square root transformations) on a single feature each time.
> > If any **error** occurs while applying a certain transformation to the feature, simple message will be printed on the console and corresponding plot becomes empty.

In [16]:
def get_estimates_of_location(data):
    
    return "Mean: {}, Median: {}, Mode: {}".format(round(data.mean(), 2), round(np.median(data), 2), round(statistics.mode(data), 2))

def apply_transformations(dataset, feature, power):
    
    fig, ax = plt.subplots(nrows = 1, ncols = 5, figsize = (30, 6))
    
    try:
        fitted_data, lmbda = stats.yeojohnson(dataset[feature])
        sns.histplot(pd.DataFrame(fitted_data, columns=[feature]), kde=True, ax = ax[0])

        ax[0].set_title(get_estimates_of_location(fitted_data))
        ax[0].axvline(x = round(fitted_data.mean(), 2), label = 'Mean', color = 'r', linestyle = '--')
        ax[0].axvline(x = round(statistics.median(fitted_data), 2), label = 'Median', color = 'green', linestyle = '--')
        ax[0].axvline(x = round(statistics.mode(fitted_data), 2), label = 'Mode', color = 'y', linestyle = '--')
    
    except:
        print('Yeojohnson Transformation Failed.')
    
    try:
        fitted_data, lmbda = stats.boxcox(dataset[feature])
        sns.histplot(pd.DataFrame(fitted_data, columns=[feature]), kde=True, ax = ax[1])

        ax[1].set_title(get_estimates_of_location(fitted_data))
        ax[1].axvline(x = round(fitted_data.mean(), 2), label = 'Mean', color = 'r', linestyle = '--')
        ax[1].axvline(x = round(statistics.median(fitted_data), 2), label = 'Median', color = 'green', linestyle = '--')
        ax[1].axvline(x = round(statistics.mode(fitted_data), 2), label = 'Mode', color = 'y', linestyle = '--')
    
    except:
        print('Boxcox Transformation Failed.')
    
    try:
        fitted_data = np.log(dataset['Age_random'])
        sns.histplot(data = fitted_data, kde=True, ax = ax[2])

        ax[2].set_title(get_estimates_of_location(fitted_data))
        ax[2].axvline(x = round(fitted_data.mean(), 2), label = 'Mean{}', color = 'r', linestyle = '--')
        ax[2].axvline(x = round(statistics.median(fitted_data), 2), label = 'Median', color = 'green', linestyle = '--')
        ax[2].axvline(x = round(statistics.mode(fitted_data), 2), label = 'Mode', color = 'y', linestyle = '--')
    
    except:
        print('Log Transformation Failed.')

    try:
        fitted_data = np.power(dataset[feature], power)
        sns.histplot(data = fitted_data, kde=True, ax = ax[3])

        ax[3].set_title(get_estimates_of_location(fitted_data))
        ax[3].axvline(x = round(fitted_data.mean(), 2), label = 'Mean', color = 'r', linestyle = '--')
        ax[3].axvline(x = round(statistics.median(fitted_data), 2), label = 'Median', color = 'green', linestyle = '--')
        ax[3].axvline(x = round(statistics.mode(fitted_data), 2), label = 'Mode', color = 'y', linestyle = '--')
    
    except:
        print('Power Transformation Failed.')

    try:
        fitted_data = np.sqrt(dataset[feature])
        sns.histplot(data = fitted_data, kde=True, ax = ax[4])

        ax[4].set_title(get_estimates_of_location(fitted_data))
        ax[4].axvline(x = round(fitted_data.mean(), 2), label = 'Mean', color = 'r', linestyle = '--')
        ax[4].axvline(x = round(statistics.median(fitted_data), 2), label = 'Median', color = 'green', linestyle = '--')
        ax[4].axvline(x = round(statistics.mode(fitted_data), 2), label = 'Mode', color = 'y', linestyle = '--')
    
    except:
        print('Square root Transformation Failed.')

> After selecting an imputation method, we can start applying transformations and see the distribution

In [17]:
apply_transformations(dataset, 'Age_random', 0.65)

### Fare

#### Applying Transformations

> After running the below piece of code, you will see a message "Boxcox Transformation Failed.". This is due to box-cox transformation **does not accepts** the value 0.

### $$
\begin{align}
\text{Box-Cox Transformation = }
\begin{cases}
\frac{x_i^\lambda - 1}{\lambda}&\text{for $\lambda \neq 0$}\\
ln(x_i)&\text{for $\lambda = 0$}\\
\end{cases}
\end{align}
$$

In [18]:
apply_transformations(dataset, 'Fare', 0.65)

In [104]:
fig = px.scatter(dataset, x = 'Fare', y = 'Age', color = 'Survived', size = (0.45 * dataset['Age_random']))
fig.update_layout(height = 800)

fig.show()

## Analysis on Target Variable

# Remapping categorical features

In [105]:
pClass_mappings = {1: 'Upper', 2: 'Middle', 3: 'Lower'}
embarked_mappings = {'C': 'Cherbourg', 'Q': 'Queenstown', 'S': 'Southampton'}

dataset['Pclass'] = dataset['Pclass'].map(pClass_mappings)
dataset['Embarked'] = dataset['Embarked'].map(embarked_mappings)

In [106]:
# Data for Bar chart
not_survived = dataset[dataset['Survived'] == 0]['Sex'].value_counts(ascending = False)
survived = dataset[dataset['Survived'] == 1]['Sex'].value_counts(ascending = False)

# Data for Pie chart
sex_data = dataset.groupby(by = ['Sex']).agg('count').sort_values(by = 'Sex', ascending = False)['Survived']

# Making subplots
fig = make_subplots(rows=1, cols=2, specs = [[{'type': 'xy'}, {'type': 'domain'}]], 
                    subplot_titles = ['Number of people survived in both genders', 'Total number of male and female percentage'])

trace0 = go.Bar(name = 'Not Survived', x = not_survived.index, y = not_survived.values)
trace1 = go.Bar(name = 'Survived', x = survived.index, y = survived.values)
trace2 = go.Pie(name = 'Gender', values = sex_data, labels = sex_data.index, pull = [.1])

data = [trace0, trace1, trace2]

fig.add_trace(trace0, row = 1, col = 1)
fig.add_trace(trace1, row = 1, col = 1)
fig.add_trace(trace2, row = 1, col = 2)

fig.update_layout({'title': {'text': 'Gender vs Survived Distribution', 'x': 0.5, 'y': 0.95, 'font_size': 26, 'font_color': 'green'}}, showlegend  = False)

pyo.iplot(fig)

> As we can see why the dataset is imbalanced. In this case, we have 64.8% of male data and only 35.2% of female data. 

> 468 out of 577 males died in the tragic incident, while 233 out of 314 females died.
> > 81.11% of male people died and 74.20% of females died.

In [107]:
pie_data = dataset['Pclass'].value_counts(ascending = True)

not_survived = dataset[dataset['Survived'] == 0]['Pclass'].value_counts(ascending = False)
survived = dataset[dataset['Survived'] == 1]['Pclass'].value_counts(ascending = False)

fig = make_subplots(rows = 1, cols = 2, specs = [[{'type': 'xy'}, {'type': 'domain'}]])

fig.add_trace(go.Bar(name = 'Not Survived', x = not_survived.index, y = not_survived.values), row = 1, col = 1)
fig.add_trace(go.Bar(name = 'Survived', x = survived.index, y = survived.values), row = 1, col = 1)
fig.add_trace(go.Pie(name = 'PClass', labels = pie_data.index, values = pie_data.values, pull = [.1, .1]), row = 1, col = 2)

fig.update_layout({'title': {'text': 'Pclass vs Survived Distribution', 'font_color': 'green', 'font_size': 26, 'x': 0.5, 'y': 0.95}}, showlegend = False)

fig.show()

> Many people belong to lower socio-economic status died when compared to other status groups.
> > According to our population, 55.1% of people belong to lower socio-economic status. Out of which, 75.76% people died in the incident.

> > People belong to upper status group were saved more than any other status groups. i.e., only 37% of people from upper class died and 52.17% of people from middle class group died.

In [108]:
pie_data = dataset['Embarked'].value_counts(ascending = False)

not_survived = dataset[dataset['Survived'] == 0]['Embarked'].value_counts(ascending = False)
survived = dataset[dataset['Survived'] == 1]['Embarked'].value_counts(ascending = False)


fig = make_subplots(rows = 1, cols = 2, specs = [[{'type': 'xy'}, {'type': 'domain'}]])

fig.add_trace(go.Bar(name = 'Not Survived', x = not_survived.index, y = not_survived.values), row = 1, col = 1)
fig.add_trace(go.Bar(name = 'Survived', x = survived.index, y = survived.values), row = 1, col = 1)
fig.add_trace(go.Pie(name = 'Embarked', values = pie_data.values, labels = pie_data.index, pull = [.1, .1]), row = 1, col = 2)

fig.update_layout({'title': {'text': 'Embarked vs Survived Distribution', 'font_color': 'green', 'font_size': 26, 'x': 0.5, 'y': 0.95}}, showlegend = False)

fig.show()

## Categorical Features

> Cardinality of categorical features

In [113]:
x_names = dataset[categorical_features].nunique().sort_values(ascending=False).index
y_values = dataset[categorical_features].nunique().sort_values(ascending=False)

data = [go.Bar(x = x_names, y = y_values)]
layout = go.Layout(width = 800, height = 500, yaxis_title = 'Number of unique categories')

fig = go.Figure(data = data, layout = layout)

pyo.iplot(fig)

# Feature Engineering 

In [114]:
dataset = pd.read_csv('../input/titanic/train.csv')

In [115]:
dataset.isna().any()

In [116]:
X_train, X_test, y_train, y_test = train_test_split(
    dataset.drop(['Survived'], axis=1),
    dataset['Survived'],
    test_size=0.3,
    random_state=0
)

## Missing Features

### Numerical Features

#### Age

In [117]:
age_imputer = RandomSampleImputer(variables = ['Age'], random_state = 1111)
age_imputer.fit(X_train, y_train)

X_train = age_imputer.transform(X_train)
X_test = age_imputer.transform(X_test)

> During data analysis part, we tried various imputers to see the distribution. Out of which I select random sample imputer.
> > Random Sample Imputer doesn't change the distribution of the feature but it draws samples from the original feature distribution and fills the missing values. Hence, the variance of the feature is not changed.

> > Note: It is important to set the seed to control the randomness of the feature distribution while using random sample imputation. 

In [118]:
age_transform = PowerTransformer(variables = ['Age'], exp = 0.65)
age_transform.fit(X_train, y_train)

X_train = age_transform.transform(X_train)
X_test = age_transform.transform(X_test)

#### Fare

In [119]:
fare_mean = X_train['Fare'].mean() # To replace the missing data in test data.

from feature_engine.transformation import YeoJohnsonTransformer

fare_yeojohnson = YeoJohnsonTransformer(variables=['Fare'])
fare_yeojohnson.fit(X_train, y_train)

X_train = fare_yeojohnson.transform(X_train)
X_test = fare_yeojohnson.transform(X_test)

In [120]:
fig = plt.figure(figsize=(11, 8))
sns.heatmap(X_train.corr(), annot = True, linewidths=0.3)

plt.show()

### Categorical Features

#### Ticket

In [121]:
def ticket_transform(transform_dataset):
    
    dataset = transform_dataset.copy()
    
    dataset['Ticket'] = dataset['Ticket'].apply(lambda x: 'alpha_numeric_ticket' if re.search('[a-zA-Z]', str(x)) else x)

    dataset['Ticket'] = dataset['Ticket'].apply(lambda x: 'three_digit_ticket' if len(str(x)) == 3 else x)
    dataset['Ticket'] = dataset['Ticket'].apply(lambda x: 'four_digit_ticket' if len(str(x)) == 4 else x)
    dataset['Ticket'] = dataset['Ticket'].apply(lambda x: 'five_digit_ticket' if len(str(x)) == 5 else x)
    dataset['Ticket'] = dataset['Ticket'].apply(lambda x: 'six_digit_ticket' if len(str(x)) == 6 else x)
    dataset['Ticket'] = dataset['Ticket'].apply(lambda x: 'seven_digit_ticket' if len(str(x)) == 7 else x)
    
    return dataset

> Instead of removing the Ticket feature from the dataset because of it's high cardinality, we can use this feature to derive new features which might help us improving our model performance. This is known as feature engineering.
> > I created 6 groups from the existing tickets names. These 6 groups are names according to their ticket names. 

> >  Note: If this feature acts as a bad predictor, during feature selection, algorithm will remove it automatically.

In [123]:
X_train = ticket_transform(X_train)
X_test = ticket_transform(X_test)

#### Name

In [124]:
def name_transform(transform_dataset):
    
    dataset = transform_dataset.copy()
    
    dataset['Name'] = dataset['Name'].apply(lambda x: 'Mr' if 'Mr.' in x else x)
    dataset['Name'] = dataset['Name'].apply(lambda x: 'Mrs' if 'Mrs.' in x else x)
    dataset['Name'] = dataset['Name'].apply(lambda x: 'Miss' if 'Miss' in x else x)
    dataset['Name'] = dataset['Name'].apply(lambda x: 'Master' if 'Master' in x else x)
    dataset['Name'] = dataset['Name'].apply(lambda x: 'Other' if len(str(x)) > 6 else x)
    
    return dataset

> Instead of dropping the name feature due to its high cardinality, we can create a new feature out of it. 
> > I created 5 groups out of it.

In [125]:
X_train = name_transform(X_train)
X_test = name_transform(X_test)

#### Cabin

> Due to it's high missing value percentage, I had to drop this column.
> > Note: In future versions of this notebook, I will try to find whether we can use this column or not.

In [126]:
dataset['Cabin'].isna().sum() / len(dataset['Cabin'])

> Cabin consists of 77% of missing values, So we drop it.

#### Embarked

In [133]:
X_train['Embarked'] = X_train['Embarked'].fillna(statistics.mode(X_train['Embarked']))
X_test['Embarked'] = X_test['Embarked'].fillna(statistics.mode(X_train['Embarked']))

> Instead of dropping the two rows, I had used mode imputation to fill the missing data.

# Feature Mapping

In [128]:
pClass_mappings = {1: 'Upper', 2: 'Middle', 3: 'Lower'}
embarked_mappings = {'C': 'Cherbourg', 'Q': 'Queenstown', 'S': 'Southampton'}

In [129]:
X_train['Pclass'] = X_train['Pclass'].map(pClass_mappings)
X_test['Pclass'] = X_test['Pclass'].map(pClass_mappings)

X_train['Embarked'] = X_train['Embarked'].map(embarked_mappings)
X_test['Embarked'] = X_test['Embarked'].map(embarked_mappings)

In [134]:
X_train.isna().any()

In [135]:
X_test.isna().any()

## Dropping Features

In [132]:
X_train = X_train.drop(['PassengerId', 'Cabin'], axis = 1)
X_test = X_test.drop(['PassengerId', 'Cabin'], axis = 1)

# One Hot Encoding

In [136]:
top_three_categories = ['Name', 'Ticket']
ohe = ['Pclass', 'Sex', 'Embarked']

In [137]:
from feature_engine.encoding import OneHotEncoder

ohc_ticket = OneHotEncoder(top_categories = 3, variables = top_three_categories)
ohc_ticket.fit(X_train, y_train)

X_train = ohc_ticket.transform(X_train)
X_test = ohc_ticket.transform(X_test)

> I created two columns using Name and Ticket features. Instead of using every group that I created I am only using top 3 groups to avoid higher dimensionality.

In [138]:
ohe = OneHotEncoder(variables = ohe)
ohe.fit(X_train, y_train)

X_train = ohe.transform(X_train)
X_test = ohe.transform(X_test)

> One hot encoding all the remaining categorical features other than Name and Ticket.

## Scaling 

In [139]:
scaler = MinMaxScaler(feature_range = (0, 1))

scaler.fit(X_train) 

X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)

In [140]:
y_train = y_train.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

## Encoding and Decoding Fns for target feature

In [142]:
def target_encode(value):
    target_dict = {0: 'Not_Survived', 1: 'Survived'}
    
    return target_dict[value]

def target_decode(value):
    target_dict = {'Not_Survived': 0, 'Survived': 1}
    
    return target_dict[value]

> These methods are used to change to values of the target from int like numbers to string. This is helpful while handling imbalanced dataset.

In [143]:
y_train = y_train.apply(target_encode)
y_test = y_test.apply(target_encode)

# Feature Selection

In [145]:
from sklearn.pipeline import Pipeline
from feature_engine.selection import RecursiveFeatureAddition, DropConstantFeatures, DropDuplicateFeatures

pipe = Pipeline([
    ('constant', DropConstantFeatures(tol = 0.998)),
    ('duplicated', DropDuplicateFeatures()),
])

pipe.fit(X_train)

# Removing the features based on the conditions in the above pipeline
X_train = pipe.transform(X_train)
X_test = pipe.transform(X_test)

model = GradientBoostingClassifier(n_estimators = 100, max_depth = 4, random_state = 10)

rfa = RecursiveFeatureAddition(variables = None, estimator = model, scoring = 'roc_auc', 
                               threshold = 0.0001, cv=2)

rfa.fit(X_train,y_train)

X_train = rfa.transform(X_train)
X_test = rfa.transform(X_test)

> Recursive Feature Addition is a feature selection technique where we select features based on the repetitive modelling on the train set deriving the importance of afeature in predicting the output.
> > Steps:

> > > Rank the features according to their importance derived from machine learning algorithm.

> > > With the most important feature obtained from above step, build a machine learning algorithm with only one feature and calculate the metric that you want to compare.

> > > In the next step add next important feature and build another machine learning model. Calculate the performance metric. If the metric increases by the arbitrary set threshold, then the feature is important and should be kept. Otherwise remove the feature and grab the next important feature and perform the same process until all features are done. 

In [146]:
rfa.feature_importances_.plot.bar(figsize=(20,6))
plt.xlabel('Features')
plt.ylabel('Importance')
plt.show()

> These are the feature that are marked as important from RecursiveFeatureAddition technique.

# Over Sampling and Under Sampling

> There are so many problems that we come across while solving imbalanced dataset. One of the major problem is that, our model will only identify the majority class alone instead of treating both classes equally. 

> > To overcome this problem we use under sampling the majority class or over sampling the minority class. The decision of using the technique is very important because of the following reasons:

> > > Under sampling the majority class means deleting the data points form the majority class to lessen the data. Because of removing the data points from the majority class we might loose important data from our dataset. 

> > > Over sampling the minority class means creating more synthetic data from the existing data from the minority class. Because of creating more synthetic data we might introduce the data which is very similar to the original data and failing to learn the important information which helps in dividing the target classes.

> > > In this notebook, I have used both the oversampling and undersampling techniques.

> NOTE: Handling the imbalanced dataset is whole other concept for researching. I just gave the introduction about the techniques used in this notebook.

In [147]:

sm = SMOTE(sampling_strategy = 0.8, random_state = 0, k_neighbors = 4, n_jobs = 4)
tl = TomekLinks(sampling_strategy = 'majority', n_jobs=4)

smtomek = SMOTETomek(sampling_strategy = 'auto', random_state = 0, smote = sm, 
                     tomek = tl, n_jobs = 4)

X_train, y_train = smtomek.fit_resample(X_train, y_train)

In [148]:
y_train = y_train.apply(target_decode)
y_test = y_test.apply(target_decode)

# Useful Functions

> These methods are used to retrive the metrics for different classification algorithms.

> Go to the Summary Metrics Tab to directly see the end results of 10 classification algorithms.

In [149]:
def print_metrics(y_train_pred, y_test_pred):
    
    print()
    
    print("Train set accuracy_score: ", accuracy_score(y_train, y_train_pred))
    print("Test set accuracy_score: ", accuracy_score(y_test, y_test_pred))
    
    print("-----------------------------------------------------------------------------------------------------------------------")
    
    print("Train set classification_report: \n\n", classification_report(y_train, y_train_pred))
    print()
    print("Test set classification_report: \n\n", classification_report(y_test, y_test_pred))

    print("-----------------------------------------------------------------------------------------------------------------------")
    
    print("Train set roc_auc_score: ", roc_auc_score(y_train, y_train_pred))
    print("Test set roc_auc_score: ", roc_auc_score(y_test, y_test_pred))
    
    print("-----------------------------------------------------------------------------------------------------------------------")
    
    print("Train set confusion_matrix: \n\n", confusion_matrix(y_train, y_train_pred))
    print()
    print("Test set confusion_matrix: \n\n", confusion_matrix(y_test, y_test_pred))

In [150]:
def get_model_score(classifier, train, test):
    
    return round(classifier.score(train, test) * 100, 2)

In [151]:
def get_precision_recall(y_train, y_train_pred, y_test, y_test_pred):
    
    train_precision = round(precision_score(y_train, y_train_pred), 2)
    train_recall = round(recall_score(y_train, y_train_pred), 2)

    test_precision = round(precision_score(y_test, y_test_pred), 2)
    test_recall = round(recall_score(y_test, y_test_pred), 2)
    
    return train_precision, train_recall, test_precision, test_recall

# Models

## GradientBoostingClassifier

In [152]:
gradient_boosting_classifier = GradientBoostingClassifier(n_estimators=100, max_depth=4, random_state=10)

gradient_boosting_classifier.fit(X_train, y_train)

In [153]:
y_train_pred = gradient_boosting_classifier.predict(X_train)
y_test_pred = gradient_boosting_classifier.predict(X_test)

In [154]:
print_metrics(y_train_pred, y_test_pred)

In [155]:
gradient_boosting_train_score = get_model_score(gradient_boosting_classifier, X_train, y_train)
gradient_boosting_test_score = get_model_score(gradient_boosting_classifier, X_test, y_test)

gradient_boosting_train_precision, gradient_boosting_train_recall, gradient_boosting_test_precision, gradient_boosting_test_recall = get_precision_recall(y_train, y_train_pred, y_test, y_test_pred)

## GP Minimize

In [ ]:
# gradient_boosting_GB_classifier = GradientBoostingClassifier(n_estimators=100, max_depth=4, random_state=10, verbose=True)

# from skopt import gp_minimize
# from skopt.plots import plot_convergence
# from skopt.space import Real, Integer, Categorical
# from skopt.utils import use_named_args
# from sklearn.model_selection import cross_val_score


# param_grid = [
#     Integer(200, 2500, name = 'n_estimators'),
#     Integer(1, 10, name = 'max_depth'),
#     Real(0.01, 0.99, name = 'learning_rate'),
#     Integer(2, 20, name = 'min_samples_split'),
#     Integer(1, 30, name = 'min_samples_leaf'),
#     Categorical(['log_loss', 'deviance', 'exponential'], name = 'loss')    
# ]

# @use_named_args(param_grid)
# def objective(**params):
    
#     # model with new parameters
#     gradient_boosting_GB_classifier.set_params(**params)

#     # optimization function (hyperparam response function)
#     value = np.mean(cross_val_score(gradient_boosting_GB_classifier, X_train, y_train,
#                                     cv=5, n_jobs=-4,scoring='accuracy'))

#     # negate because we need to minimize
#     return -value


# gp_ = gp_minimize(objective, param_grid, n_initial_points=10, acq_func='gp_hedge', 
#                   n_calls=100, random_state=0, 
# )

# print("Best score=%.4f" % gp_.fun)

In [ ]:
# plot_convergence(gp_)

In [156]:
gradient_boosting_GB_classifier = GradientBoostingClassifier(n_estimators=2202, max_depth=6, learning_rate = 0.01, min_samples_split = 18, min_samples_leaf = 8, loss = 'deviance')
gradient_boosting_GB_classifier.fit(X_train, y_train)

In [157]:
y_train_pred = gradient_boosting_GB_classifier.predict(X_train)
y_test_ored = gradient_boosting_GB_classifier.predict(X_test)

In [158]:
print_metrics(y_train_pred, y_test_pred)

In [159]:
gradient_boosting_GB_train_score = get_model_score(gradient_boosting_GB_classifier, X_train, y_train)
gradient_boosting_GB_test_score = get_model_score(gradient_boosting_GB_classifier, X_test, y_test)

gradient_boosting_GB_train_precision, gradient_boosting_GB_train_recall, gradient_boosting_GB_test_precision, gradient_boosting_GB_test_recall = get_precision_recall(y_train, y_train_pred, y_test, y_test_pred)

## AdaBoost Classifier

In [160]:
classifier_adaboost = AdaBoostClassifier()
classifier_adaboost.fit(X_train, y_train)

In [161]:
y_train_pred = classifier_adaboost.predict(X_train)
y_test_pred = classifier_adaboost.predict(X_test)

In [162]:
print_metrics(y_train_pred, y_test_pred)

In [163]:
adabost_train_score = get_model_score(classifier_adaboost, X_train, y_train)
adabost_test_score = get_model_score(classifier_adaboost, X_test, y_test)

adaboost_train_precision, adaboost_train_recall, adaboost_test_precision, adaboost_test_recall = get_precision_recall(y_train, y_train_pred, y_test, y_test_pred)

In [164]:
classifier = LogisticRegression(C = 1/0.001, max_iter=4000)
classifier.fit(X_train, y_train)

In [165]:
y_train_pred = classifier.predict(X_train)
y_test_pred = classifier.predict(X_test)

In [166]:
print_metrics(y_train_pred, y_test_pred)

In [167]:
logit_train_score = get_model_score(classifier, X_train, y_train)
logit_test_score = get_model_score(classifier, X_test, y_test)

logit_train_precision, logit_train_recall, logit_test_precision, logit_test_recall = get_precision_recall(y_train, y_train_pred, y_test, y_test_pred)

## XGBoost

In [168]:
xgboost_classifier = xgboost.XGBClassifier(use_label_encoder = False, eval_metric = 'mlogloss')
xgboost_classifier.fit(X_train, y_train)

In [169]:
y_train_pred = xgboost_classifier.predict(X_train)
y_test_pred = xgboost_classifier.predict(X_test)

In [170]:
print_metrics(y_train_pred, y_test_pred)

In [171]:
xgboost_train_score = get_model_score(xgboost_classifier, X_train, y_train)
xgboost_test_score = get_model_score(xgboost_classifier, X_test, y_test)

xgboost_train_precision, xgboost_train_recall, xgboost_test_precision, xgboost_test_recall = get_precision_recall(y_train, y_train_pred, y_test, y_test_pred)

### Grid Search

In [172]:
# estimator = xgboost.XGBClassifier(
#     objective= 'binary:logistic',
#     nthread=4,
#     seed=909
# )

# parameters = {
#     'max_depth': range (2, 20, 1),
#     'n_estimators': range(60, 400, 10),
#     'learning_rate': [0.1, 0.01, 0.01, 0.2, 0.02, 0.002, 0.3, 0.03, 0.003]
# }

# xgboost_grid_search = GridSearchCV(
#     estimator = estimator,
#     param_grid = parameters,
#     scoring = 'accuracy',
#     n_jobs = 10,
#     cv = 5,
#     verbose=True
# )

# xgboost_grid_search.fit(X_train, y_train)

In [173]:
# xgboost_grid_search.best_params_ # {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 140}

In [174]:
xgboost_grid_search = xgboost.XGBClassifier(learning_rate = 0.01, max_depth = 5, n_estimators = 140)
xgboost_grid_search.fit(X_train, y_train)

In [175]:
y_train_pred = xgboost_grid_search.predict(X_train)
y_test_pred = xgboost_grid_search.predict(X_test)

In [176]:
print_metrics(y_train_pred, y_test_pred)

In [177]:
xgboost_grid_train_score = get_model_score(xgboost_grid_search, X_train, y_train)
xgboost_grid_test_score = get_model_score(xgboost_grid_search, X_test, y_test)

xgboost_grid_train_precision, xgboost_grid_train_recall, xgboost_grid_test_precision, xgboost_grid_test_recall = get_precision_recall(y_train, y_train_pred, y_test, y_test_pred)

## KNN

In [178]:
knn_classifier = KNeighborsClassifier(n_neighbors = 3, weights = 'distance', metric = 'minkowski', p = 1)
knn_classifier.fit(X_train, y_train)

In [179]:
y_train_pred = knn_classifier.predict(X_train)
y_test_pred = knn_classifier.predict(X_test)

print_metrics(y_train_pred, y_test_pred)

In [180]:
knn_train_score = get_model_score(knn_classifier, X_train, y_train)
knn_test_score = get_model_score(knn_classifier, X_test, y_test)

knn_train_precision, knn_train_recall, knn_test_precision, knn_test_recall = get_precision_recall(y_train, y_train_pred, y_test, y_test_pred)

## Navie Bayes

In [181]:
from sklearn.naive_bayes import GaussianNB
navie_bayes_classifier = GaussianNB()
navie_bayes_classifier.fit(X_train, y_train)

In [182]:
y_train_pred = navie_bayes_classifier.predict(X_train)
y_test_pred = navie_bayes_classifier.predict(X_test)

print_metrics(y_train_pred, y_test_pred)

In [183]:
navie_bayes_train_score = get_model_score(navie_bayes_classifier, X_train, y_train)
navie_bayes_test_score = get_model_score(navie_bayes_classifier, X_test, y_test)

navie_bayes_train_precision, navie_bayes_train_recall, navie_bayes_test_precision, navie_bayes_test_recall = get_precision_recall(y_train, y_train_pred, y_test, y_test_pred)

## SVC

In [184]:
svc_classifier = SVC(kernel = 'poly', random_state = 0, C = 10)
svc_classifier.fit(X_train, y_train)

In [185]:
y_train_pred = svc_classifier.predict(X_train)
y_test_pred = svc_classifier.predict(X_test)

print_metrics(pd.Series(y_train_pred), pd.Series(y_test_pred))

In [186]:
svc_train_score = round(svc_classifier.score(X_train, y_train) * 100, 2)
svc_test_score = round(svc_classifier.score(X_test, y_test) * 100, 2)

In [187]:
svc_train_score = get_model_score(svc_classifier, X_train, y_train)
svc_test_score = get_model_score(svc_classifier, X_test, y_test)

svc_train_precision, svc_train_recall, svc_test_precision, svc_test_recall = get_precision_recall(y_train, y_train_pred, y_test, y_test_pred)

## Decision Tree

In [188]:
decision_tree_classifier = DecisionTreeClassifier(random_state=192)
decision_tree_classifier.fit(X_train, y_train)

In [189]:
plt.figure(figsize =(80,20))

plot_tree(decision_tree_classifier, feature_names=X_train.columns, max_depth=2, filled=True)

plt.show()

In [190]:
y_train_pred = decision_tree_classifier.predict(X_train)
y_test_pred = decision_tree_classifier.predict(X_test)

In [191]:
print_metrics(y_train_pred, y_test_pred)

In [192]:
decision_tree_train_score = get_model_score(decision_tree_classifier, X_train, y_train)
decision_tree_test_score = get_model_score(decision_tree_classifier, X_test, y_test)

decision_tree_train_precision, decision_tree_train_recall, decision_tree_test_precision, decision_tree_test_recall = get_precision_recall(y_train, y_train_pred, y_test, y_test_pred)

### Grid Search

In [193]:
decision_tree_classifier = DecisionTreeClassifier(random_state=192)

parameters = {
    'max_depth': range (2, 20, 1),
    'min_samples_split': [2, 3, 4, 6, 7, 8],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

decision_tree_grid_search = GridSearchCV(
    estimator = decision_tree_classifier,
    param_grid = parameters,
    scoring = 'accuracy',
    n_jobs = 10,
    cv = 10,
    verbose=True
)

decision_tree_grid_search.fit(X_train, y_train)

In [194]:
print(decision_tree_grid_search.best_estimator_)

In [195]:
decision_tree_grid_search = decision_tree_grid_search.best_estimator_
decision_tree_grid_search.fit(X_train, y_train)

y_train_pred = decision_tree_grid_search.predict(X_train)
y_test_pred = decision_tree_grid_search.predict(X_test)

In [196]:
print_metrics(y_train_pred, y_test_pred)

In [197]:
decision_tree_grid_search_train_score = get_model_score(decision_tree_grid_search, X_train, y_train)
decision_tree_grid_search_test_score = get_model_score(decision_tree_grid_search, X_test, y_test)

decision_tree_grid_search_train_precision, decision_tree_grid_search_train_recall, decision_tree_grid_search_test_precision, decision_tree_grid_search_test_recall = get_precision_recall(y_train, y_train_pred, y_test, y_test_pred)

# Summary Metrics

In [198]:
models = pd.DataFrame({
    
    'Model': ['GradientBoosting_GP_Classifier', 'GradientBoosting_Classifier', 'AdaBoost_Classifier', 'Logistic_Regression', 'XGBoost', 'XGBoost_GridSearch', 
              'KNN', 'Navie_Bayes', 'SVC', 'Decision_Tree_Classifier', 'Decision_Tree_Classifier_GridSearch'],
    
    'Train_set_accuracy': [gradient_boosting_GB_train_score, gradient_boosting_train_score, adabost_train_score, logit_train_score, xgboost_train_score, xgboost_grid_train_score, 
                           knn_train_score, navie_bayes_train_score, svc_train_score, decision_tree_train_score, decision_tree_grid_search_train_score],
    
    'Test_set_accuracy': [gradient_boosting_test_score, gradient_boosting_test_score, adabost_test_score, logit_test_score, xgboost_test_score, xgboost_grid_test_score, 
                          knn_test_score, navie_bayes_test_score, svc_test_score, decision_tree_test_score, decision_tree_grid_search_test_score],
    
    'Train_precision': [gradient_boosting_GB_train_precision, gradient_boosting_train_precision, adaboost_train_precision, logit_train_precision, xgboost_train_precision, xgboost_grid_train_precision, knn_train_precision, 
                        navie_bayes_train_precision, svc_train_precision, decision_tree_train_precision, decision_tree_grid_search_train_precision],
    
    'Test_precision': [gradient_boosting_test_precision, gradient_boosting_test_precision, adaboost_test_precision, logit_test_precision, xgboost_test_precision, xgboost_grid_test_precision, knn_test_precision, 
                       navie_bayes_test_precision, svc_test_precision, decision_tree_test_precision, decision_tree_grid_search_test_precision],
    
    'Train_recall': [gradient_boosting_GB_train_recall, gradient_boosting_train_recall, adaboost_train_recall, logit_train_recall, xgboost_train_recall, xgboost_grid_train_recall, knn_train_recall,
                     navie_bayes_train_recall, svc_train_recall, decision_tree_train_recall, decision_tree_grid_search_train_recall],
    
    'Test_recall': [gradient_boosting_GB_test_recall, gradient_boosting_test_recall, adaboost_test_recall, logit_test_recall, xgboost_test_recall, xgboost_grid_test_recall, knn_test_recall,
                    navie_bayes_test_recall, svc_test_recall, decision_tree_test_recall, decision_tree_grid_search_test_recall]
})

In [199]:
models.sort_values(by = 'Train_set_accuracy', ascending = True).sort_values(by = 'Test_set_accuracy', ascending = False).reset_index().drop(['index'], axis = 1)